# Kapitola 14: Vizualizace AI v bludisti - Animovany A* algoritmus

**Blok 2: Prohledavaci algoritmy a optimalizace**

---

## Co se naucite

V teto kapitole:
- Vytvorime **nahodne generovane bludiste** pomoci DFS algoritmu
- Implementujeme **animovanou vizualizaci A*** v realnem case
- Porovname vizualne **BFS vs A*** - jak "premysli" AI
- Pochopime **Open Set vs Closed Set** a jejich vyznam
- Vytvorime **interaktivni projekt** pro Google Colab

---

## Motivace: Proc vizualizovat algoritmy?

V minulych kapitolach jsme se naucili BFS, DFS a A* algoritmy. Ale videt je v akci
je neco uplne jineho nez jen cist kod! Vizualizace nam ukaze:

1. **Jak AI "premysli"** - ktere cesty zvazuje, kam se diva
2. **Efektivitu algoritmu** - kolik prace dela zbytecne
3. **Rozdil mezi algoritmy** - BFS prozkoumava vsechno, A* jde "chytre"

In [ ]:
# Instalace a import knihoven
!pip install matplotlib numpy ipywidgets -q

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display, clear_output
import heapq
import random
import time
from collections import deque

# Pro animace v Colabu
from matplotlib import rc
rc('animation', html='jshtml')

print("Knihovny uspesne nacteny!")
print("Pripraveno pro vizualizaci AI v bludisti.")

---

## Cast 1: Generovani nahodneho bludiste

### Proc nahodne bludiste?

Rucne vytvarene bludiste je nudne. Kazdy test by vypadal stejne.
Nahodne generovani zajisti:
- Pokazde jiny problem k reseni
- Moznost testovat algoritmy na ruznych strukturach
- Realistictejsi scenar pro AI

### Algoritmus generovani: Randomized DFS

1. Zacneme s mrizkou plnou zdi
2. Vybereme startovni bod
3. Nahodne "prorazime" cestu ke sousedum
4. Pouzijeme zasobnik (DFS) pro navrat z cest

In [ ]:
def generuj_bludiste(vyska, sirka, seed=None):
    """
    Generuje nahodne bludiste pomoci Randomized DFS (Recursive Backtracking).
    
    Args:
        vyska: Pocet radku bludiste (musi byt lichy)
        sirka: Pocet sloupcu bludiste (musi byt lichy)
        seed: Seed pro opakovatelnost
    
    Returns:
        2D numpy pole (0 = cesta, 1 = zed)
    """
    if seed is not None:
        random.seed(seed)
    
    # Zajistime liche rozmery
    if vyska % 2 == 0:
        vyska += 1
    if sirka % 2 == 0:
        sirka += 1
    
    # Vytvorime mrizku plnou zdi
    bludiste = np.ones((vyska, sirka), dtype=int)
    
    # Startovni pozice (musi byt na lichych souradnicich)
    start_r, start_c = 1, 1
    bludiste[start_r, start_c] = 0
    
    # Zasobnik pro DFS
    zasobnik = [(start_r, start_c)]
    
    # Smery: nahoru, dolu, vlevo, vpravo (po 2 polich)
    smery = [(-2, 0), (2, 0), (0, -2), (0, 2)]
    
    while zasobnik:
        aktualni_r, aktualni_c = zasobnik[-1]
        
        # Najdeme nenavstivene sousedy
        sousedi = []
        for dr, dc in smery:
            novy_r, novy_c = aktualni_r + dr, aktualni_c + dc
            # Kontrola hranic a zda je soused jeste zed
            if (0 < novy_r < vyska - 1 and 
                0 < novy_c < sirka - 1 and 
                bludiste[novy_r, novy_c] == 1):
                sousedi.append((novy_r, novy_c, dr, dc))
        
        if sousedi:
            # Vybereme nahodneho souseda
            novy_r, novy_c, dr, dc = random.choice(sousedi)
            
            # Prorazime zed mezi aktualnim a novym
            bludiste[aktualni_r + dr // 2, aktualni_c + dc // 2] = 0
            bludiste[novy_r, novy_c] = 0
            
            zasobnik.append((novy_r, novy_c))
        else:
            # Zadny nenavstiveny soused - vratime se
            zasobnik.pop()
    
    return bludiste

# Testujeme generovani
test_bludiste = generuj_bludiste(15, 21, seed=42)
print(f"Vygenerovano bludiste {test_bludiste.shape[0]}x{test_bludiste.shape[1]}")
print(f"Pocet cest: {np.sum(test_bludiste == 0)}")
print(f"Pocet zdi: {np.sum(test_bludiste == 1)}")

In [ ]:
def zobraz_bludiste(bludiste, start=None, cil=None, cesta=None, title="Bludiste"):
    """
    Zobrazi bludiste s volitelnym startem, cilem a cestou.
    """
    vyska, sirka = bludiste.shape
    
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Barevna mapa: bila = cesta, cerna = zed
    ax.imshow(bludiste, cmap='binary', interpolation='nearest')
    
    # Oznacime start (zelena)
    if start:
        ax.add_patch(patches.Rectangle(
            (start[1] - 0.5, start[0] - 0.5), 1, 1,
            facecolor='green', edgecolor='darkgreen', linewidth=2
        ))
        ax.text(start[1], start[0], 'S', ha='center', va='center', 
                fontsize=12, fontweight='bold', color='white')
    
    # Oznacime cil (cervena)
    if cil:
        ax.add_patch(patches.Rectangle(
            (cil[1] - 0.5, cil[0] - 0.5), 1, 1,
            facecolor='red', edgecolor='darkred', linewidth=2
        ))
        ax.text(cil[1], cil[0], 'C', ha='center', va='center', 
                fontsize=12, fontweight='bold', color='white')
    
    # Vykreslime cestu (modra)
    if cesta:
        for r, c in cesta:
            if (r, c) != start and (r, c) != cil:
                ax.add_patch(patches.Rectangle(
                    (c - 0.5, r - 0.5), 1, 1,
                    facecolor='dodgerblue', alpha=0.7
                ))
    
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.set_xticks([])
    ax.set_yticks([])
    plt.tight_layout()
    plt.show()

# Zobrazime bludiste se startem a cilem
start = (1, 1)
cil = (test_bludiste.shape[0] - 2, test_bludiste.shape[1] - 2)

zobraz_bludiste(test_bludiste, start, cil, title="Nase nahodne vygenerovane bludiste")

---

## Cast 2: Implementace algoritmu s krokovym zaznamem

### Proc potrebujeme zaznam kroku?

Pro animaci potrebujeme vedet, co algoritmus delal v kazdem okamziku:
- Ktery uzel prave zpracovava (aktualni)
- Ktere uzly jsou v **Open Set** (kandidati k prozkoumani)
- Ktere uzly jsou v **Closed Set** (uz prozkoumane)
- Jakou cestu aktualne zvazuje

### Terminologie

| Pojem | Vyznam |
|-------|--------|
| **Open Set** | Uzly, ktere jsme objevili, ale jeste neprozkoumali |
| **Closed Set** | Uzly, ktere jsme uz plne prozkoumali |
| **g(n)** | Skutecna cena cesty od startu k uzlu n |
| **h(n)** | Heuristicky odhad ceny od n do cile |
| **f(n)** | Celkova priorita: f(n) = g(n) + h(n) |

In [ ]:
def manhattan_vzdalenost(a, b):
    """Manhattanska vzdalenost mezi dvema body."""
    return abs(a[0] - b[0]) + abs(a[1] - b[1])


def a_star_s_kroky(bludiste, start, cil):
    """
    A* algoritmus, ktery zaznamenava kazdy krok pro animaci.
    
    Returns:
        kroky: Seznam slovniku s informacemi o kazdem kroku
        cesta: Nalezena cesta (nebo None)
    """
    vyska, sirka = bludiste.shape
    
    # Smery pohybu: nahoru, dolu, vlevo, vpravo
    smery = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    
    # Prioritni fronta: (f_score, g_score, pozice, cesta)
    # g_score pouzijeme jako sekundarni kriterium pro stabilitu
    open_set = []
    h_start = manhattan_vzdalenost(start, cil)
    heapq.heappush(open_set, (h_start, 0, start, [start]))
    
    # Mnoziny pro sledovani
    open_set_pozice = {start}  # Pozice v open set
    closed_set = set()          # Uz prozkoumane pozice
    g_scores = {start: 0}       # Nejlepsi g score pro kazdou pozici
    
    # Zaznam kroku pro animaci
    kroky = []
    
    while open_set:
        # Vyjmeme uzel s nejnizsi prioritou
        f, g, aktualni, cesta = heapq.heappop(open_set)
        open_set_pozice.discard(aktualni)
        
        # Preskocime, pokud jsme uz nasli lepsi cestu
        if aktualni in closed_set:
            continue
        
        # Pridame do closed set
        closed_set.add(aktualni)
        
        # Zaznamename krok
        kroky.append({
            'aktualni': aktualni,
            'open_set': set(open_set_pozice),
            'closed_set': set(closed_set),
            'cesta': list(cesta),
            'f_score': f,
            'g_score': g
        })
        
        # Zkontrolujeme, zda jsme v cili
        if aktualni == cil:
            return kroky, cesta
        
        # Prozkoumame sousedy
        for dr, dc in smery:
            soused = (aktualni[0] + dr, aktualni[1] + dc)
            
            # Kontrola hranic
            if not (0 <= soused[0] < vyska and 0 <= soused[1] < sirka):
                continue
            
            # Kontrola zdi
            if bludiste[soused[0], soused[1]] == 1:
                continue
            
            # Preskocime uz prozkoumane
            if soused in closed_set:
                continue
            
            # Vypocitame nove skore
            novy_g = g + 1
            
            # Zkontrolujeme, zda je to lepsi cesta
            if soused not in g_scores or novy_g < g_scores[soused]:
                g_scores[soused] = novy_g
                h = manhattan_vzdalenost(soused, cil)
                f = novy_g + h
                
                heapq.heappush(open_set, (f, novy_g, soused, cesta + [soused]))
                open_set_pozice.add(soused)
    
    # Cesta neexistuje
    return kroky, None


# Testujeme
kroky_astar, cesta_astar = a_star_s_kroky(test_bludiste, start, cil)
print(f"A* dokoncil za {len(kroky_astar)} kroku")
print(f"Delka cesty: {len(cesta_astar) if cesta_astar else 'nenalezeno'}")
print(f"Celkem prozkoumano uzlu: {len(kroky_astar[-1]['closed_set'])}")

In [ ]:
def bfs_s_kroky(bludiste, start, cil):
    """
    BFS algoritmus, ktery zaznamenava kazdy krok pro animaci.
    
    Returns:
        kroky: Seznam slovniku s informacemi o kazdem kroku
        cesta: Nalezena cesta (nebo None)
    """
    vyska, sirka = bludiste.shape
    smery = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    
    # Fronta pro BFS
    fronta = deque([(start, [start])])
    open_set_pozice = {start}
    closed_set = set()
    
    kroky = []
    
    while fronta:
        aktualni, cesta = fronta.popleft()
        open_set_pozice.discard(aktualni)
        
        if aktualni in closed_set:
            continue
        
        closed_set.add(aktualni)
        
        # Zaznamename krok
        kroky.append({
            'aktualni': aktualni,
            'open_set': set(open_set_pozice),
            'closed_set': set(closed_set),
            'cesta': list(cesta)
        })
        
        if aktualni == cil:
            return kroky, cesta
        
        for dr, dc in smery:
            soused = (aktualni[0] + dr, aktualni[1] + dc)
            
            if not (0 <= soused[0] < vyska and 0 <= soused[1] < sirka):
                continue
            if bludiste[soused[0], soused[1]] == 1:
                continue
            if soused in closed_set or soused in open_set_pozice:
                continue
            
            fronta.append((soused, cesta + [soused]))
            open_set_pozice.add(soused)
    
    return kroky, None


# Testujeme
kroky_bfs, cesta_bfs = bfs_s_kroky(test_bludiste, start, cil)
print(f"BFS dokoncil za {len(kroky_bfs)} kroku")
print(f"Delka cesty: {len(cesta_bfs) if cesta_bfs else 'nenalezeno'}")
print(f"Celkem prozkoumano uzlu: {len(kroky_bfs[-1]['closed_set'])}")

---

## Cast 3: Animovana vizualizace

Nyni vytvorime animaci, ktera krok za krokem ukaze, jak algoritmus premysli.

### Barevne oznaceni

- **Zelena**: Start
- **Cervena**: Cil
- **Zluta**: Aktualne zpracovavany uzel
- **Svetle modra**: Open Set (kandidati k prozkoumani)
- **Seda**: Closed Set (uz prozkoumane)
- **Modra cesta**: Finalni nalezena cesta

In [ ]:
def vytvor_animaci(bludiste, kroky, start, cil, cesta_final, nazev="Algoritmus", interval=100):
    """
    Vytvori animaci prohledavani bludiste.
    
    Args:
        bludiste: 2D pole bludiste
        kroky: Seznam kroku z algoritmu
        start: Startovni pozice
        cil: Cilova pozice
        cesta_final: Finalni cesta
        nazev: Nazev algoritmu
        interval: Casovy interval mezi snimky (ms)
    """
    vyska, sirka = bludiste.shape
    
    fig, ax = plt.subplots(figsize=(12, 8))
    
    def animuj(frame):
        ax.clear()
        
        # Zakladni bludiste
        ax.imshow(bludiste, cmap='binary', interpolation='nearest')
        
        if frame < len(kroky):
            krok = kroky[frame]
            
            # Closed set (seda)
            for pos in krok['closed_set']:
                if pos != start and pos != cil:
                    ax.add_patch(patches.Rectangle(
                        (pos[1] - 0.5, pos[0] - 0.5), 1, 1,
                        facecolor='lightgray', alpha=0.8
                    ))
            
            # Open set (svetle modra)
            for pos in krok['open_set']:
                if pos != start and pos != cil:
                    ax.add_patch(patches.Rectangle(
                        (pos[1] - 0.5, pos[0] - 0.5), 1, 1,
                        facecolor='lightskyblue', alpha=0.8
                    ))
            
            # Aktualni uzel (zluta)
            aktualni = krok['aktualni']
            if aktualni != start and aktualni != cil:
                ax.add_patch(patches.Rectangle(
                    (aktualni[1] - 0.5, aktualni[0] - 0.5), 1, 1,
                    facecolor='yellow', edgecolor='orange', linewidth=2
                ))
            
            titulek = f"{nazev} - Krok {frame + 1}/{len(kroky)}"
        else:
            # Finalni cesta
            if cesta_final:
                for pos in cesta_final:
                    if pos != start and pos != cil:
                        ax.add_patch(patches.Rectangle(
                            (pos[1] - 0.5, pos[0] - 0.5), 1, 1,
                            facecolor='dodgerblue', alpha=0.8
                        ))
            titulek = f"{nazev} - HOTOVO! Cesta nalezena ({len(cesta_final)} kroku)"
        
        # Start a cil
        ax.add_patch(patches.Rectangle(
            (start[1] - 0.5, start[0] - 0.5), 1, 1,
            facecolor='green', edgecolor='darkgreen', linewidth=2
        ))
        ax.text(start[1], start[0], 'S', ha='center', va='center', 
                fontsize=10, fontweight='bold', color='white')
        
        ax.add_patch(patches.Rectangle(
            (cil[1] - 0.5, cil[0] - 0.5), 1, 1,
            facecolor='red', edgecolor='darkred', linewidth=2
        ))
        ax.text(cil[1], cil[0], 'C', ha='center', va='center', 
                fontsize=10, fontweight='bold', color='white')
        
        ax.set_title(titulek, fontsize=14, fontweight='bold')
        ax.set_xticks([])
        ax.set_yticks([])
        
        # Legenda
        legenda_text = "Zelena=Start | Cervena=Cil | Zluta=Aktualni | Modra=Kandidati | Seda=Prozkoumane"
        ax.text(0.5, -0.05, legenda_text, transform=ax.transAxes, 
                ha='center', fontsize=9, style='italic')
    
    # Pridame extra snimky na konec pro zobrazeni vysledku
    pocet_snimku = len(kroky) + 5
    
    anim = FuncAnimation(fig, animuj, frames=pocet_snimku, 
                         interval=interval, repeat=False)
    plt.close(fig)
    
    return anim

print("Funkce pro animaci pripravena!")

In [ ]:
# Vytvorime mensí bludiste pro prehlednejsi animaci
male_bludiste = generuj_bludiste(11, 15, seed=123)
male_start = (1, 1)
male_cil = (male_bludiste.shape[0] - 2, male_bludiste.shape[1] - 2)

# Spustime A*
kroky_a, cesta_a = a_star_s_kroky(male_bludiste, male_start, male_cil)

print(f"A* naslo cestu za {len(kroky_a)} kroku")
print(f"Delka cesty: {len(cesta_a)}")
print("\nGeneruji animaci A*...")

# Vytvorime a zobrazime animaci
anim_astar = vytvor_animaci(male_bludiste, kroky_a, male_start, male_cil, 
                             cesta_a, "A* Algoritmus", interval=200)
HTML(anim_astar.to_jshtml())

In [ ]:
# Pro porovnani - BFS
kroky_b, cesta_b = bfs_s_kroky(male_bludiste, male_start, male_cil)

print(f"BFS naslo cestu za {len(kroky_b)} kroku")
print(f"Delka cesty: {len(cesta_b)}")
print("\nGeneruji animaci BFS...")

anim_bfs = vytvor_animaci(male_bludiste, kroky_b, male_start, male_cil, 
                          cesta_b, "BFS Algoritmus", interval=150)
HTML(anim_bfs.to_jshtml())

---

## Cast 4: Porovnani BFS vs A*

Pojdme vytvorit vizualni porovnani obou algoritmu vedle sebe.

In [ ]:
def porovnej_algoritmy(bludiste, start, cil):
    """
    Porovna BFS a A* na stejnem bludisti.
    """
    # Spustime oba algoritmy
    kroky_bfs, cesta_bfs = bfs_s_kroky(bludiste, start, cil)
    kroky_astar, cesta_astar = a_star_s_kroky(bludiste, start, cil)
    
    # Vytvorime porovnavaci graf
    fig, axes = plt.subplots(1, 2, figsize=(16, 8))
    
    for ax, kroky, cesta, nazev, barva in [
        (axes[0], kroky_bfs, cesta_bfs, "BFS", "orange"),
        (axes[1], kroky_astar, cesta_astar, "A*", "green")
    ]:
        # Zakladni bludiste
        ax.imshow(bludiste, cmap='binary', interpolation='nearest')
        
        # Vsechny prozkoumane uzly (seda)
        if kroky:
            for pos in kroky[-1]['closed_set']:
                if pos != start and pos != cil:
                    ax.add_patch(patches.Rectangle(
                        (pos[1] - 0.5, pos[0] - 0.5), 1, 1,
                        facecolor='lightgray', alpha=0.7
                    ))
        
        # Finalni cesta
        if cesta:
            for pos in cesta:
                if pos != start and pos != cil:
                    ax.add_patch(patches.Rectangle(
                        (pos[1] - 0.5, pos[0] - 0.5), 1, 1,
                        facecolor=barva, alpha=0.8
                    ))
        
        # Start a cil
        ax.add_patch(patches.Rectangle(
            (start[1] - 0.5, start[0] - 0.5), 1, 1,
            facecolor='green', edgecolor='darkgreen', linewidth=2
        ))
        ax.add_patch(patches.Rectangle(
            (cil[1] - 0.5, cil[0] - 0.5), 1, 1,
            facecolor='red', edgecolor='darkred', linewidth=2
        ))
        
        prozkoumano = len(kroky[-1]['closed_set']) if kroky else 0
        delka = len(cesta) if cesta else 0
        
        ax.set_title(f"{nazev}\nProzkoumano: {prozkoumano} uzlu | Delka cesty: {delka}", 
                     fontsize=12, fontweight='bold')
        ax.set_xticks([])
        ax.set_yticks([])
    
    plt.suptitle("Porovnani BFS vs A*", fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Statistiky
    print("\n" + "="*50)
    print("POROVNANI STATISTIK")
    print("="*50)
    print(f"{'Metrika':<25} {'BFS':<15} {'A*':<15}")
    print("-"*50)
    print(f"{'Prozkoumano uzlu':<25} {len(kroky_bfs[-1]['closed_set']):<15} {len(kroky_astar[-1]['closed_set']):<15}")
    print(f"{'Delka cesty':<25} {len(cesta_bfs):<15} {len(cesta_astar):<15}")
    
    uspora = len(kroky_bfs[-1]['closed_set']) - len(kroky_astar[-1]['closed_set'])
    procenta = (uspora / len(kroky_bfs[-1]['closed_set'])) * 100 if kroky_bfs else 0
    print(f"{'A* uspora':<25} {uspora} uzlu ({procenta:.1f}% mene prace)")
    print("="*50)


# Vytvorime vetsi bludiste pro lepsi porovnani
velke_bludiste = generuj_bludiste(21, 31, seed=42)
velky_start = (1, 1)
velky_cil = (velke_bludiste.shape[0] - 2, velke_bludiste.shape[1] - 2)

porovnej_algoritmy(velke_bludiste, velky_start, velky_cil)

---

## Cast 5: Interaktivni experimenty

### Experiment 1: Vliv velikosti bludiste

In [ ]:
def benchmark_velikosti():
    """
    Testuje efektivitu algoritmu na ruznych velikostech bludiste.
    """
    velikosti = [11, 21, 31, 41, 51]
    vysledky = []
    
    for vel in velikosti:
        bludiste = generuj_bludiste(vel, vel, seed=42)
        start = (1, 1)
        cil = (vel - 2, vel - 2)
        
        # BFS
        t_start = time.time()
        kroky_b, cesta_b = bfs_s_kroky(bludiste, start, cil)
        cas_bfs = time.time() - t_start
        
        # A*
        t_start = time.time()
        kroky_a, cesta_a = a_star_s_kroky(bludiste, start, cil)
        cas_astar = time.time() - t_start
        
        vysledky.append({
            'velikost': vel,
            'bfs_uzly': len(kroky_b[-1]['closed_set']),
            'astar_uzly': len(kroky_a[-1]['closed_set']),
            'bfs_cas': cas_bfs,
            'astar_cas': cas_astar
        })
        print(f"Velikost {vel}x{vel}: BFS={len(kroky_b[-1]['closed_set'])} uzlu, A*={len(kroky_a[-1]['closed_set'])} uzlu")
    
    return vysledky

vysledky = benchmark_velikosti()

# Vizualizace vysledku
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

velikosti = [v['velikost'] for v in vysledky]
bfs_uzly = [v['bfs_uzly'] for v in vysledky]
astar_uzly = [v['astar_uzly'] for v in vysledky]

# Graf 1: Pocet prozkoumanych uzlu
ax1 = axes[0]
x = range(len(velikosti))
width = 0.35
ax1.bar([i - width/2 for i in x], bfs_uzly, width, label='BFS', color='orange')
ax1.bar([i + width/2 for i in x], astar_uzly, width, label='A*', color='green')
ax1.set_xlabel('Velikost bludiste')
ax1.set_ylabel('Pocet prozkoumanych uzlu')
ax1.set_title('Efektivita: BFS vs A*')
ax1.set_xticks(x)
ax1.set_xticklabels([f"{v}x{v}" for v in velikosti])
ax1.legend()
ax1.grid(axis='y', alpha=0.3)

# Graf 2: Uspora A* oproti BFS
ax2 = axes[1]
uspora = [(b - a) / b * 100 for b, a in zip(bfs_uzly, astar_uzly)]
ax2.bar(x, uspora, color='dodgerblue')
ax2.set_xlabel('Velikost bludiste')
ax2.set_ylabel('Uspora A* oproti BFS (%)')
ax2.set_title('Procentualni uspora prace diky A*')
ax2.set_xticks(x)
ax2.set_xticklabels([f"{v}x{v}" for v in velikosti])
ax2.grid(axis='y', alpha=0.3)

for i, u in enumerate(uspora):
    ax2.text(i, u + 1, f"{u:.1f}%", ha='center', fontsize=10)

plt.tight_layout()
plt.show()

### Experiment 2: Ruzne heuristiky

In [ ]:
def euklidova_vzdalenost(a, b):
    """Euklidovska vzdalenost (vzdusna cara)."""
    return ((a[0] - b[0])**2 + (a[1] - b[1])**2) ** 0.5

def chebyshev_vzdalenost(a, b):
    """Chebyshevova vzdalenost (povoleny diagonalni pohyb)."""
    return max(abs(a[0] - b[0]), abs(a[1] - b[1]))

def nulova_heuristika(a, b):
    """Zadna heuristika - A* se chova jako Dijkstra/BFS."""
    return 0


def a_star_custom_heuristika(bludiste, start, cil, heuristika):
    """
    A* s volitelnou heuristikou.
    """
    vyska, sirka = bludiste.shape
    smery = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    
    open_set = []
    h_start = heuristika(start, cil)
    heapq.heappush(open_set, (h_start, 0, start, [start]))
    
    closed_set = set()
    g_scores = {start: 0}
    
    while open_set:
        f, g, aktualni, cesta = heapq.heappop(open_set)
        
        if aktualni in closed_set:
            continue
        
        closed_set.add(aktualni)
        
        if aktualni == cil:
            return cesta, len(closed_set)
        
        for dr, dc in smery:
            soused = (aktualni[0] + dr, aktualni[1] + dc)
            
            if not (0 <= soused[0] < vyska and 0 <= soused[1] < sirka):
                continue
            if bludiste[soused[0], soused[1]] == 1:
                continue
            if soused in closed_set:
                continue
            
            novy_g = g + 1
            
            if soused not in g_scores or novy_g < g_scores[soused]:
                g_scores[soused] = novy_g
                h = heuristika(soused, cil)
                f = novy_g + h
                heapq.heappush(open_set, (f, novy_g, soused, cesta + [soused]))
    
    return None, len(closed_set)


# Testujeme ruzne heuristiky
heuristiky = [
    ("Manhattan", manhattan_vzdalenost),
    ("Euklidova", euklidova_vzdalenost),
    ("Chebyshev", chebyshev_vzdalenost),
    ("Zadna (Dijkstra)", nulova_heuristika)
]

test_maze = generuj_bludiste(31, 41, seed=42)
test_start = (1, 1)
test_cil = (test_maze.shape[0] - 2, test_maze.shape[1] - 2)

print("POROVNANI HEURISTIK")
print("="*60)
print(f"{'Heuristika':<20} {'Prozkoumano uzlu':<20} {'Delka cesty':<15}")
print("-"*60)

vysledky_h = []
for nazev, heur in heuristiky:
    cesta, uzly = a_star_custom_heuristika(test_maze, test_start, test_cil, heur)
    delka = len(cesta) if cesta else 0
    vysledky_h.append((nazev, uzly, delka))
    print(f"{nazev:<20} {uzly:<20} {delka:<15}")

print("="*60)
print("\nVsechny heuristiky nasly stejne dlouhou cestu (to je spravne!).")
print("Rozdil je v tom, kolik prace musely udelat.")

---

## Cast 6: Mini-projekt - Vlastni bludiste

### Ukol: Vytvorte vlastni bludiste a sledujte AI

Muzete si nakreslit vlastni bludiste rucne a sledovat, jak ho AI resi!

In [ ]:
# Vlastni bludiste - upravte podle libosti!
# 0 = cesta, 1 = zed

vlastni_bludiste = np.array([
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
    [1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1],
    [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1],
    [1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1],
    [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
    [1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1],
    [1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
])

vlastni_start = (1, 1)
vlastni_cil = (9, 11)

# Zobrazime a porovname
print("Vase vlastni bludiste:")
zobraz_bludiste(vlastni_bludiste, vlastni_start, vlastni_cil, title="Vlastni bludiste")

# Porovnani
porovnej_algoritmy(vlastni_bludiste, vlastni_start, vlastni_cil)

In [ ]:
# Animace na vlastnim bludisti
kroky_v, cesta_v = a_star_s_kroky(vlastni_bludiste, vlastni_start, vlastni_cil)

print(f"A* naslo cestu za {len(kroky_v)} kroku")
print("Generuji animaci...")

anim_vlastni = vytvor_animaci(vlastni_bludiste, kroky_v, vlastni_start, vlastni_cil,
                               cesta_v, "A* na vasem bludisti", interval=300)
HTML(anim_vlastni.to_jshtml())

---

## Cviceni k procviceni

### Cviceni 1: Vetsi bludiste
Vygenerujte bludiste 51x51 a porovnejte BFS vs A*. O kolik procent je A* efektivnejsi?

### Cviceni 2: Vlastni prekazky
Upravte `vlastni_bludiste` tak, aby A* musel prozkoumat vice uzlu. Kde umistit zdi?

### Cviceni 3: Diagonalni pohyb
Upravte algoritmus tak, aby AI mohla chodit i diagonalne (8 smeru misto 4).

### Cviceni 4: Vice cilu
Upravte algoritmus tak, aby nasel cestu pres vice kontrolnich bodu (A -> B -> C -> D).

In [ ]:
# Prostor pro vase reseni cviceni 1
# Vygenerujte bludiste 51x51 a spocitejte uspornost A*

# VAS KOD ZDE:
# velke = generuj_bludiste(51, 51, seed=...)
# ...

---

## Shrnuti kapitoly

### Co jsme se naucili:

1. **Generovani bludiste** - Randomized DFS pro vytvareni nahodnych bludist
2. **Zaznam kroku** - Jak sledovat prubeh algoritmu pro animaci
3. **Animovana vizualizace** - Matplotlib + IPython pro interaktivni zobrazeni
4. **Open Set vs Closed Set** - Pochopeni vnitrniho fungovani A*
5. **Porovnani algoritmu** - BFS vs A* na ruznych bludistich
6. **Vliv heuristiky** - Jak volba heuristiky ovlivnuje efektivitu

### Klicove poznatky:

| Aspekt | BFS | A* |
|--------|-----|----|
| Prozkoumane uzly | Vice | Mene |
| Zarucena optimalita | Ano | Ano (s primerene heuristikou) |
| Pametova narocnost | Vyssi | Nizsi |
| Vhodne pouziti | Male problemy | Velke problemy |

### Dalsi kroky:

V dalsi kapitole se podivame na **Geneticke algoritmy** - jak se AI muze "vyvíjet" k lepsim resenim inspirovanim prirodou!

---

*Kapitola 14 - Vizualizace AI v bludisti*